In [1]:
import os
import json
import PIL.Image
import random
import numpy as np

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import torchvision

from src.data.PascalVOC import dataset_voc, prepare_dataloaders_pascal_voc
from src.attacks.attacks import FastGradientSign, ProjectedGradientDescent
from src.training.Trainer import Trainer

In [2]:
def seed_everything(seed_value=4995):
    os.environ['PYTHONHASHSEED'] = str(seed_value)
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    torch.cuda.manual_seed_all(seed_value)

    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    
seed_everything()

In [3]:
root_dir = 'Data/PascalVOC/VOCdevkit/VOC2012'

In [4]:
config = {
    'lr': 0.005,
    'batch_size': 64,
    'weight_decay': 0.01
}

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
def load_model(num_classes = 20, model_path = None, to_cuda = True):
    if not model_path:
        model = torchvision.models.resnet18(pretrained = True)
        input_feat = model.fc.in_features
        
        model.fc = nn.Linear(input_feat, num_classes)
        model.fc.reset_parameters()
        loaded_state_dict = False
    
    else:
        print("Loaded", model_path)
        model = torchvision.models.resnet18()
        input_feat = model.fc.in_features
        model.fc = nn.Linear(input_feat, num_classes)
        loaded_model = torch.load(model_path)
        model.load_state_dict(loaded_model['model_state_dict'])
        loaded_state_dict = True
        
    if to_cuda:
        model = model.to('cuda')
        
    return model, loaded_state_dict

# Universal attack

In [8]:
robust_resnet, loaded_state_dict = load_model(model_path = 'models/pascal_voc_resnet_adv_fine_Tune_small_ball_msls.pt')

dataloaders, classes = prepare_dataloaders_pascal_voc(root_dir, batch_size = config['batch_size'], smaller_resize =True)

loss_fn = nn.BCEWithLogitsLoss(weight=None, reduction='mean')

train_params = [p for p in robust_resnet.parameters() if p.requires_grad]

optimizer = torch.optim.Adam(train_params, lr = config['lr'], weight_decay = config['weight_decay'])
scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[4,6], gamma=0.1)

training_params = {
    'dataloaders': dataloaders,
    'optimizer': optimizer,
    'scheduler': scheduler
}

class UniversalAttack(nn.Module):
    def __init__(self, vector = None):
        super(UniversalAttack, self).__init__()
        self.vector = vector
        
    def forward(self, image, labels, random_start = True, compute_original_prediction = False, compute_new_preds = False):
        return image + self.vector
        
vec = torch.load('models/uni_pert_pascal_voc.pt')
attack = UniversalAttack(vec)

trainer = Trainer(
        model = robust_resnet,
        loss_fn = loss_fn,
        classes = classes,
        num_epochs = 7,
        model_name = f'pascal_voc_resnet',
        training_params = training_params,
        multi_label = True,
        print_frequency = 10,
        adversarial_training = False,
        adversarial_attack = attack
    )
trainer.run_validation_loop_only()

trainer = Trainer(
        model = robust_resnet,
        loss_fn = loss_fn,
        classes = classes,
        num_epochs = 7,
        model_name = f'pascal_voc_resnet',
        training_params = training_params,
        multi_label = True,
        print_frequency = 10,
        adversarial_training = True,
        adversarial_attack = attack
    )
trainer.run_validation_loop_only()

Loaded models/pascal_voc_resnet_adv_fine_Tune_small_ball_msls.pt
Val Loop:
Batch: 0 of 91. Loss: 0.10000192373991013. Mean so far: 0.10000192373991013. Mean of 100: 0.10000192373991013
Batch: 10 of 91. Loss: 0.1270599216222763. Mean so far: 0.11915948241949081. Mean of 100: 0.11915948241949081
Batch: 20 of 91. Loss: 0.15993991494178772. Mean so far: 0.12277954036281222. Mean of 100: 0.12277954036281222
Batch: 30 of 91. Loss: 0.15330980718135834. Mean so far: 0.13501583953057567. Mean of 100: 0.13501583953057567
Batch: 40 of 91. Loss: 0.12953168153762817. Mean so far: 0.13248891110827282. Mean of 100: 0.13248891110827282
Batch: 50 of 91. Loss: 0.16901348531246185. Mean so far: 0.1405193796929191. Mean of 100: 0.1405193796929191
Batch: 60 of 91. Loss: 0.11176550388336182. Mean so far: 0.13815817859817725. Mean of 100: 0.13815817859817725
Batch: 70 of 91. Loss: 0.05652468279004097. Mean so far: 0.13349662548009777. Mean of 100: 0.13349662548009777
Batch: 80 of 91. Loss: 0.0916876867413520

In [9]:
non_robust_resnet, loaded_state_dict = load_model(model_path = 'models/pascal_voc_resnet_fine_tuned.pt')

dataloaders, classes = prepare_dataloaders_pascal_voc(root_dir, batch_size = config['batch_size'], smaller_resize =True)

loss_fn = nn.BCEWithLogitsLoss(weight=None, reduction='mean')

train_params = [p for p in non_robust_resnet.parameters() if p.requires_grad]

optimizer = torch.optim.Adam(train_params, lr = config['lr'], weight_decay = config['weight_decay'])
scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[4,6], gamma=0.1)

training_params = {
    'dataloaders': dataloaders,
    'optimizer': optimizer,
    'scheduler': scheduler
}


        
vec = torch.load('models/uni_pert_pascal_voc.pt')
attack = UniversalAttack(vec)

trainer = Trainer(
        model = non_robust_resnet,
        loss_fn = loss_fn,
        classes = classes,
        num_epochs = 7,
        model_name = f'pascal_voc_resnet',
        training_params = training_params,
        multi_label = True,
        print_frequency = 10,
        adversarial_training = False,
        adversarial_attack = attack
    )
trainer.run_validation_loop_only()

trainer = Trainer(
        model = non_robust_resnet,
        loss_fn = loss_fn,
        classes = classes,
        num_epochs = 7,
        model_name = f'pascal_voc_resnet',
        training_params = training_params,
        multi_label = True,
        print_frequency = 10,
        adversarial_training = True,
        adversarial_attack = attack
    )
trainer.run_validation_loop_only()

Loaded models/pascal_voc_resnet_fine_tuned.pt
Val Loop:
Batch: 0 of 91. Loss: 0.07511293143033981. Mean so far: 0.07511293143033981. Mean of 100: 0.07511293143033981
Batch: 10 of 91. Loss: 0.07656233012676239. Mean so far: 0.08470477129925381. Mean of 100: 0.08470477129925381
Batch: 20 of 91. Loss: 0.12316854298114777. Mean so far: 0.08299414884476435. Mean of 100: 0.08299414884476435
Batch: 30 of 91. Loss: 0.11158295720815659. Mean so far: 0.09474506926152014. Mean of 100: 0.09474506926152014
Batch: 40 of 91. Loss: 0.06912567466497421. Mean so far: 0.09016162575018115. Mean of 100: 0.09016162575018115
Batch: 50 of 91. Loss: 0.09732342511415482. Mean so far: 0.09659966461214364. Mean of 100: 0.09659966461214364
Batch: 60 of 91. Loss: 0.06426148861646652. Mean so far: 0.09396961001587696. Mean of 100: 0.09396961001587696
Batch: 70 of 91. Loss: 0.04682765156030655. Mean so far: 0.090752332405725. Mean of 100: 0.090752332405725
Batch: 80 of 91. Loss: 0.06796424835920334. Mean so far: 0.08

# Robust model

In [6]:
resnet, loaded_state_dict = load_model(model_path = 'models/pascal_voc_resnet_adv_fine_Tune_small_ball_msls.pt')

for name, param in resnet.named_parameters():
    if 'fc' not in name:
        param.requires_grad = False

Loaded models/pascal_voc_resnet_fine_tuned.pt


In [7]:
dataloaders, classes = prepare_dataloaders_pascal_voc(root_dir, batch_size = config['batch_size'])

loss_fn = nn.BCEWithLogitsLoss(weight=None, reduction='mean')

train_params = [p for p in resnet.parameters() if p.requires_grad]

optimizer = torch.optim.Adam(train_params, lr = config['lr'], weight_decay = config['weight_decay'])
scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[4,6], gamma=0.1)

training_params = {
    'dataloaders': dataloaders,
    'optimizer': optimizer,
    'scheduler': scheduler
}

In [16]:
pgd = ProjectedGradientDescent(resnet, loss_fn, iterations = 5, alpha = 2/255, epsilon = 2/255, return_logits=False)


In [17]:
trainer = Trainer(
        model = resnet,
        loss_fn = loss_fn,
        classes = classes,
        num_epochs = 7,
        model_name = f'pascal_voc_resnet',
        training_params = training_params,
        multi_label = True,
        print_frequency = 10,
        adversarial_training = True,
        adversarial_attack = pgd
    )
trainer.run_validation_loop_only()

Val Loop:
Batch: 0 of 91. Loss: 0.11876717954874039. Mean so far: 0.11876717954874039. Mean of 100: 0.11876717954874039
Batch: 10 of 91. Loss: 0.14295044541358948. Mean so far: 0.13868466967886145. Mean of 100: 0.13868466967886145
Batch: 20 of 91. Loss: 0.17662620544433594. Mean so far: 0.14004749059677124. Mean of 100: 0.14004749059677124
Batch: 30 of 91. Loss: 0.17261944711208344. Mean so far: 0.15272864555158921. Mean of 100: 0.15272864555158921
Batch: 40 of 91. Loss: 0.14535491168498993. Mean so far: 0.14934259435025657. Mean of 100: 0.14934259435025657
Batch: 50 of 91. Loss: 0.18414326012134552. Mean so far: 0.15749329854460323. Mean of 100: 0.15749329854460323
Batch: 60 of 91. Loss: 0.13422195613384247. Mean so far: 0.15550654674651193. Mean of 100: 0.15550654674651193
Batch: 70 of 91. Loss: 0.06533575057983398. Mean so far: 0.1511300781243284. Mean of 100: 0.1511300781243284
Batch: 80 of 91. Loss: 0.1013786569237709. Mean so far: 0.14499337676866556. Mean of 100: 0.1449933767686

In [10]:
pgd = ProjectedGradientDescent(resnet, loss_fn, iterations = 5, alpha = 0.02, epsilon = 0.01, return_logits=False)
trainer = Trainer(
        model = resnet,
        loss_fn = loss_fn,
        classes = classes,
        num_epochs = 7,
        model_name = f'pascal_voc_resnet',
        training_params = training_params,
        multi_label = True,
        print_frequency = 10,
        adversarial_training = True,
        adversarial_attack = pgd
    )
trainer.run_validation_loop_only()

Val Loop:
Batch: 0 of 91. Loss: 0.14654956758022308. Mean so far: 0.14654956758022308. Mean of 100: 0.14654956758022308
Batch: 10 of 91. Loss: 0.1785043329000473. Mean so far: 0.16890050199898807. Mean of 100: 0.16890050199898807
Batch: 20 of 91. Loss: 0.2066251039505005. Mean so far: 0.17077774448054178. Mean of 100: 0.17077774448054178
Batch: 30 of 91. Loss: 0.2032638043165207. Mean so far: 0.18353216926897725. Mean of 100: 0.18353216926897725
Batch: 40 of 91. Loss: 0.18618370592594147. Mean so far: 0.1815333613535253. Mean of 100: 0.1815333613535253
Batch: 50 of 91. Loss: 0.21509309113025665. Mean so far: 0.19031592736057207. Mean of 100: 0.19031592736057207
Batch: 60 of 91. Loss: 0.16777831315994263. Mean so far: 0.18844358256605806. Mean of 100: 0.18844358256605806
Batch: 70 of 91. Loss: 0.08141390234231949. Mean so far: 0.18378777487177245. Mean of 100: 0.18378777487177245
Batch: 80 of 91. Loss: 0.11995601654052734. Mean so far: 0.17621417435598963. Mean of 100: 0.176214174355989

In [11]:
pgd = ProjectedGradientDescent(resnet, loss_fn, iterations = 5, alpha = 0.05, epsilon = 0.03, return_logits=False)
trainer = Trainer(
        model = resnet,
        loss_fn = loss_fn,
        classes = classes,
        num_epochs = 10,
        model_name = f'pascal_voc_resnet',
        training_params = training_params,
        multi_label = True,
        print_frequency = 10,
        adversarial_training = True,
        adversarial_attack = pgd
    )
trainer.run_validation_loop_only()

Val Loop:
Batch: 0 of 91. Loss: 0.23902259767055511. Mean so far: 0.23902259767055511. Mean of 100: 0.23902259767055511
Batch: 10 of 91. Loss: 0.28413909673690796. Mean so far: 0.26571876217018475. Mean of 100: 0.26571876217018475
Batch: 20 of 91. Loss: 0.32277533411979675. Mean so far: 0.26955048739910126. Mean of 100: 0.26955048739910126
Batch: 30 of 91. Loss: 0.3004283607006073. Mean so far: 0.28462809468469313. Mean of 100: 0.28462809468469313
Batch: 40 of 91. Loss: 0.3291986584663391. Mean so far: 0.28820465468778844. Mean of 100: 0.28820465468778844
Batch: 50 of 91. Loss: 0.31992703676223755. Mean so far: 0.29992283442441153. Mean of 100: 0.29992283442441153
Batch: 60 of 91. Loss: 0.2688003480434418. Mean so far: 0.2974076246628996. Mean of 100: 0.2974076246628996
Batch: 70 of 91. Loss: 0.13726957142353058. Mean so far: 0.2911385239010126. Mean of 100: 0.2911385239010126
Batch: 80 of 91. Loss: 0.18153999745845795. Mean so far: 0.2791480271168697. Mean of 100: 0.2791480271168697
B

In [12]:
pgd = ProjectedGradientDescent(resnet, loss_fn, iterations = 5, alpha = 8/255, epsilon = 4/255, return_logits=False)
trainer = Trainer(
        model = resnet,
        loss_fn = loss_fn,
        classes = classes,
        num_epochs = 10,
        model_name = f'pascal_voc_resnet',
        training_params = training_params,
        multi_label = True,
        print_frequency = 10,
        adversarial_training = True,
        adversarial_attack = pgd
    )
trainer.run_validation_loop_only()

Val Loop:
Batch: 0 of 91. Loss: 0.17669706046581268. Mean so far: 0.17669706046581268. Mean of 100: 0.17669706046581268
Batch: 10 of 91. Loss: 0.21546335518360138. Mean so far: 0.20123421468517996. Mean of 100: 0.20123421468517996
Batch: 20 of 91. Loss: 0.2428436279296875. Mean so far: 0.20382427956376756. Mean of 100: 0.20382427956376756
Batch: 30 of 91. Loss: 0.23585255444049835. Mean so far: 0.21708822538775782. Mean of 100: 0.21708822538775782
Batch: 40 of 91. Loss: 0.23357973992824554. Mean so far: 0.21698525620669853. Mean of 100: 0.21698525620669853
Batch: 50 of 91. Loss: 0.2495107650756836. Mean so far: 0.22663406472580105. Mean of 100: 0.22663406472580105
Batch: 60 of 91. Loss: 0.20285038650035858. Mean so far: 0.2247024549812567. Mean of 100: 0.2247024549812567
Batch: 70 of 91. Loss: 0.09962891787290573. Mean so far: 0.21963076870626128. Mean of 100: 0.21963076870626128
Batch: 80 of 91. Loss: 0.13988171517848969. Mean so far: 0.21053649485111237. Mean of 100: 0.21053649485111

In [13]:
trainer = Trainer(
        model = resnet,
        loss_fn = loss_fn,
        classes = classes,
        num_epochs = 10,
        model_name = f'pascal_voc_resnet',
        training_params = training_params,
        multi_label = True,
        print_frequency = 10
    )
trainer.run_validation_loop_only()

Val Loop:
Batch: 0 of 91. Loss: 0.1031193882226944. Mean so far: 0.1031193882226944. Mean of 100: 0.1031193882226944
Batch: 10 of 91. Loss: 0.12257780879735947. Mean so far: 0.12136885523796082. Mean of 100: 0.12136885523796082
Batch: 20 of 91. Loss: 0.16107255220413208. Mean so far: 0.12253246172553017. Mean of 100: 0.12253246172553017
Batch: 30 of 91. Loss: 0.15509480237960815. Mean so far: 0.13534338604057988. Mean of 100: 0.13534338604057988
Batch: 40 of 91. Loss: 0.12370319664478302. Mean so far: 0.1314016820454016. Mean of 100: 0.1314016820454016
Batch: 50 of 91. Loss: 0.16722027957439423. Mean so far: 0.13927834057340435. Mean of 100: 0.13927834057340435
Batch: 60 of 91. Loss: 0.1149883046746254. Mean so far: 0.13716014889908618. Mean of 100: 0.13716014889908618
Batch: 70 of 91. Loss: 0.05679671838879585. Mean so far: 0.13293152087381188. Mean of 100: 0.13293152087381188
Batch: 80 of 91. Loss: 0.09081918746232986. Mean so far: 0.12761285192804572. Mean of 100: 0.1276128519280457

# Larger ball

In [21]:
resnet, loaded_state_dict = load_model(model_path = 'models/pascal_voc_resnet_adv_fine_Tune_large_ball_msml.pt')

for name, param in resnet.named_parameters():
    if 'fc' not in name:
        param.requires_grad = False

Loaded models/pascal_voc_resnet_adv_fine_Tune_large_ball_msml.pt


In [22]:
dataloaders, classes = prepare_dataloaders_pascal_voc(root_dir, batch_size = config['batch_size'])

loss_fn = nn.BCEWithLogitsLoss(weight=None, reduction='mean')

train_params = [p for p in resnet.parameters() if p.requires_grad]

optimizer = torch.optim.Adam(train_params, lr = config['lr'], weight_decay = config['weight_decay'])
scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[4,6], gamma=0.1)

training_params = {
    'dataloaders': dataloaders,
    'optimizer': optimizer,
    'scheduler': scheduler
}

In [23]:
pgd = ProjectedGradientDescent(resnet, loss_fn, iterations = 5, alpha = 2/255, epsilon = 2/255, return_logits=False)
trainer = Trainer(
        model = resnet,
        loss_fn = loss_fn,
        classes = classes,
        num_epochs = 7,
        model_name = f'pascal_voc_resnet',
        training_params = training_params,
        multi_label = True,
        print_frequency = 10,
        adversarial_training = True,
        adversarial_attack = pgd
    )
trainer.run_validation_loop_only()

Val Loop:
Batch: 0 of 91. Loss: 0.18299658596515656. Mean so far: 0.18299658596515656. Mean of 100: 0.18299658596515656
Batch: 10 of 91. Loss: 0.21027831733226776. Mean so far: 0.21778703413226388. Mean of 100: 0.21778703413226388
Batch: 20 of 91. Loss: 0.2610393166542053. Mean so far: 0.2188895351829983. Mean of 100: 0.2188895351829983
Batch: 30 of 91. Loss: 0.24662180244922638. Mean so far: 0.23360221712819992. Mean of 100: 0.23360221712819992
Batch: 40 of 91. Loss: 0.2016642838716507. Mean so far: 0.2258575547759126. Mean of 100: 0.2258575547759126
Batch: 50 of 91. Loss: 0.2623445689678192. Mean so far: 0.23648501903403038. Mean of 100: 0.23648501903403038
Batch: 60 of 91. Loss: 0.1912636011838913. Mean so far: 0.2311380893969145. Mean of 100: 0.2311380893969145
Batch: 70 of 91. Loss: 0.11268346756696701. Mean so far: 0.2282836118214567. Mean of 100: 0.2282836118214567
Batch: 80 of 91. Loss: 0.1478928029537201. Mean so far: 0.21776874510594357. Mean of 100: 0.21776874510594357
Batch

In [24]:
pgd = ProjectedGradientDescent(resnet, loss_fn, iterations = 5, alpha = 0.02, epsilon = 0.01, return_logits=False)
trainer = Trainer(
        model = resnet,
        loss_fn = loss_fn,
        classes = classes,
        num_epochs = 7,
        model_name = f'pascal_voc_resnet',
        training_params = training_params,
        multi_label = True,
        print_frequency = 10,
        adversarial_training = True,
        adversarial_attack = pgd
    )
trainer.run_validation_loop_only()

Val Loop:
Batch: 0 of 91. Loss: 0.1867278516292572. Mean so far: 0.1867278516292572. Mean of 100: 0.1867278516292572
Batch: 10 of 91. Loss: 0.21144941449165344. Mean so far: 0.2201706902547316. Mean of 100: 0.2201706902547316
Batch: 20 of 91. Loss: 0.2623497545719147. Mean so far: 0.22087122287069047. Mean of 100: 0.22087122287069047
Batch: 30 of 91. Loss: 0.24925296008586884. Mean so far: 0.23563507631901773. Mean of 100: 0.23563507631901773
Batch: 40 of 91. Loss: 0.20390187203884125. Mean so far: 0.2279362216955278. Mean of 100: 0.2279362216955278
Batch: 50 of 91. Loss: 0.26306086778640747. Mean so far: 0.23843926863343107. Mean of 100: 0.23843926863343107
Batch: 60 of 91. Loss: 0.1926068365573883. Mean so far: 0.23297299298106647. Mean of 100: 0.23297299298106647
Batch: 70 of 91. Loss: 0.11326701939105988. Mean so far: 0.22999181671881339. Mean of 100: 0.22999181671881339
Batch: 80 of 91. Loss: 0.14864201843738556. Mean so far: 0.21935450708792534. Mean of 100: 0.21935450708792534
B

In [25]:
pgd = ProjectedGradientDescent(resnet, loss_fn, iterations = 5, alpha = 0.05, epsilon = 0.03, return_logits=False)
trainer = Trainer(
        model = resnet,
        loss_fn = loss_fn,
        classes = classes,
        num_epochs = 10,
        model_name = f'pascal_voc_resnet',
        training_params = training_params,
        multi_label = True,
        print_frequency = 10,
        adversarial_training = True,
        adversarial_attack = pgd
    )
trainer.run_validation_loop_only()

Val Loop:
Batch: 0 of 91. Loss: 0.19628548622131348. Mean so far: 0.19628548622131348. Mean of 100: 0.19628548622131348
Batch: 10 of 91. Loss: 0.2142484188079834. Mean so far: 0.2260778776623986. Mean of 100: 0.2260778776623986
Batch: 20 of 91. Loss: 0.26559922099113464. Mean so far: 0.22578160961469015. Mean of 100: 0.22578160961469015
Batch: 30 of 91. Loss: 0.2553713917732239. Mean so far: 0.2405965462807686. Mean of 100: 0.2405965462807686
Batch: 40 of 91. Loss: 0.2095985859632492. Mean so far: 0.23301592033083846. Mean of 100: 0.23301592033083846
Batch: 50 of 91. Loss: 0.26476070284843445. Mean so far: 0.24322853953230614. Mean of 100: 0.24322853953230614
Batch: 60 of 91. Loss: 0.1959572285413742. Mean so far: 0.23748288193687064. Mean of 100: 0.23748288193687064
Batch: 70 of 91. Loss: 0.11463995277881622. Mean so far: 0.23417565961119155. Mean of 100: 0.23417565961119155
Batch: 80 of 91. Loss: 0.15040233731269836. Mean so far: 0.22323134348348336. Mean of 100: 0.22323134348348336


In [26]:
pgd = ProjectedGradientDescent(resnet, loss_fn, iterations = 5, alpha = 8/255, epsilon = 4/255, return_logits=False)
trainer = Trainer(
        model = resnet,
        loss_fn = loss_fn,
        classes = classes,
        num_epochs = 10,
        model_name = f'pascal_voc_resnet',
        training_params = training_params,
        multi_label = True,
        print_frequency = 10,
        adversarial_training = True,
        adversarial_attack = pgd
    )
trainer.run_validation_loop_only()

Val Loop:
Batch: 0 of 91. Loss: 0.19025687873363495. Mean so far: 0.19025687873363495. Mean of 100: 0.19025687873363495
Batch: 10 of 91. Loss: 0.2125156670808792. Mean so far: 0.2223874940113588. Mean of 100: 0.2223874940113588
Batch: 20 of 91. Loss: 0.2635711133480072. Mean so far: 0.22271493999730974. Mean of 100: 0.22271493999730974
Batch: 30 of 91. Loss: 0.251632422208786. Mean so far: 0.23751355371167582. Mean of 100: 0.23751355371167582
Batch: 40 of 91. Loss: 0.20601359009742737. Mean so far: 0.2298584958402122. Mean of 100: 0.2298584958402122
Batch: 50 of 91. Loss: 0.26371365785598755. Mean so far: 0.2402488700315064. Mean of 100: 0.2402488700315064
Batch: 60 of 91. Loss: 0.193866565823555. Mean so far: 0.23467511524919604. Mean of 100: 0.23467511524919604
Batch: 70 of 91. Loss: 0.11379537731409073. Mean so far: 0.23157363259036776. Mean of 100: 0.23157363259036776
Batch: 80 of 91. Loss: 0.1493206024169922. Mean so far: 0.22082151923282647. Mean of 100: 0.22082151923282647
Batch

In [27]:
trainer = Trainer(
        model = resnet,
        loss_fn = loss_fn,
        classes = classes,
        num_epochs = 10,
        model_name = f'pascal_voc_resnet',
        training_params = training_params,
        multi_label = True,
        print_frequency = 10
    )
trainer.run_validation_loop_only()

Val Loop:
Batch: 0 of 91. Loss: 0.18060153722763062. Mean so far: 0.18060153722763062. Mean of 100: 0.18060153722763062
Batch: 10 of 91. Loss: 0.20950590074062347. Mean so far: 0.21623865311796014. Mean of 100: 0.21623865311796014
Batch: 20 of 91. Loss: 0.2601967751979828. Mean so far: 0.21760342589446477. Mean of 100: 0.21760342589446477
Batch: 30 of 91. Loss: 0.2448846846818924. Mean so far: 0.23227782691678694. Mean of 100: 0.23227782691678694
Batch: 40 of 91. Loss: 0.2002316564321518. Mean so far: 0.22450455441707518. Mean of 100: 0.22450455441707518
Batch: 50 of 91. Loss: 0.261874794960022. Mean so far: 0.23521431141039906. Mean of 100: 0.23521431141039906
Batch: 60 of 91. Loss: 0.19040000438690186. Mean so far: 0.22994685783737995. Mean of 100: 0.22994685783737995
Batch: 70 of 91. Loss: 0.1122966781258583. Mean so far: 0.22717313751788207. Mean of 100: 0.22717313751788207
Batch: 80 of 91. Loss: 0.14739690721035004. Mean so far: 0.21673699698330443. Mean of 100: 0.2167369969833044

# Non-robust model

In [14]:
resnet, loaded_state_dict = load_model(model_path = 'models/pascal_voc_resnet_fine_tuned.pt')

for name, param in resnet.named_parameters():
    if 'fc' not in name:
        param.requires_grad = False

Loaded models/pascal_voc_resnet_fine_tuned.pt


In [15]:
dataloaders, classes = prepare_dataloaders_pascal_voc(root_dir, batch_size = config['batch_size'])

loss_fn = nn.BCEWithLogitsLoss(weight=None, reduction='mean')

train_params = [p for p in resnet.parameters() if p.requires_grad]

optimizer = torch.optim.Adam(train_params, lr = config['lr'], weight_decay = config['weight_decay'])
scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[4,6], gamma=0.1)

training_params = {
    'dataloaders': dataloaders,
    'optimizer': optimizer,
    'scheduler': scheduler
}

In [16]:
pgd = ProjectedGradientDescent(resnet, loss_fn, iterations = 5, alpha = 2/255, epsilon = 2/255, return_logits=False)
trainer = Trainer(
        model = resnet,
        loss_fn = loss_fn,
        classes = classes,
        num_epochs = 7,
        model_name = f'pascal_voc_resnet',
        training_params = training_params,
        multi_label = True,
        print_frequency = 10,
        adversarial_training = True,
        adversarial_attack = pgd
    )
trainer.run_validation_loop_only()

Val Loop:
Batch: 0 of 91. Loss: 0.26751312613487244. Mean so far: 0.26751312613487244. Mean of 100: 0.26751312613487244
Batch: 10 of 91. Loss: 0.29877951741218567. Mean so far: 0.27183098820122803. Mean of 100: 0.27183098820122803
Batch: 20 of 91. Loss: 0.3478533625602722. Mean so far: 0.2784814941031592. Mean of 100: 0.2784814941031592
Batch: 30 of 91. Loss: 0.2960622310638428. Mean so far: 0.2932326452385995. Mean of 100: 0.2932326452385995
Batch: 40 of 91. Loss: 0.3941527009010315. Mean so far: 0.3020271386315183. Mean of 100: 0.3020271386315183
Batch: 50 of 91. Loss: 0.3633590638637543. Mean so far: 0.3187414083410712. Mean of 100: 0.3187414083410712
Batch: 60 of 91. Loss: 0.373209685087204. Mean so far: 0.3255806142678026. Mean of 100: 0.3255806142678026
Batch: 70 of 91. Loss: 0.20528320968151093. Mean so far: 0.3191047854406733. Mean of 100: 0.3191047854406733
Batch: 80 of 91. Loss: 0.24035556614398956. Mean so far: 0.31097739033492994. Mean of 100: 0.31097739033492994
Batch: 90 

In [17]:
pgd = ProjectedGradientDescent(resnet, loss_fn, iterations = 5, alpha = 0.02, epsilon = 0.01, return_logits=False)
trainer = Trainer(
        model = resnet,
        loss_fn = loss_fn,
        classes = classes,
        num_epochs = 7,
        model_name = f'pascal_voc_resnet',
        training_params = training_params,
        multi_label = True,
        print_frequency = 10,
        adversarial_training = True,
        adversarial_attack = pgd
    )
trainer.run_validation_loop_only()

Val Loop:
Batch: 0 of 91. Loss: 0.6244817972183228. Mean so far: 0.6244817972183228. Mean of 100: 0.6244817972183228
Batch: 10 of 91. Loss: 0.7045232057571411. Mean so far: 0.6490932323715903. Mean of 100: 0.6490932323715903
Batch: 20 of 91. Loss: 0.788415789604187. Mean so far: 0.6684551182247344. Mean of 100: 0.6684551182247344
Batch: 30 of 91. Loss: 0.6750659346580505. Mean so far: 0.6845188852279417. Mean of 100: 0.6845188852279417
Batch: 40 of 91. Loss: 0.9246464967727661. Mean so far: 0.7096428522249547. Mean of 100: 0.7096428522249547
Batch: 50 of 91. Loss: 0.8157331347465515. Mean so far: 0.7343851994065678. Mean of 100: 0.7343851994065678
Batch: 60 of 91. Loss: 0.8990116119384766. Mean so far: 0.7534157154990024. Mean of 100: 0.7534157154990024
Batch: 70 of 91. Loss: 0.5244208574295044. Mean so far: 0.7422662436122626. Mean of 100: 0.7422662436122626
Batch: 80 of 91. Loss: 0.5989598035812378. Mean so far: 0.7250093744124895. Mean of 100: 0.7250093744124895
Batch: 90 of 91. Los

In [18]:
pgd = ProjectedGradientDescent(resnet, loss_fn, iterations = 5, alpha = 0.05, epsilon = 0.03, return_logits=False)
trainer = Trainer(
        model = resnet,
        loss_fn = loss_fn,
        classes = classes,
        num_epochs = 10,
        model_name = f'pascal_voc_resnet',
        training_params = training_params,
        multi_label = True,
        print_frequency = 10,
        adversarial_training = True,
        adversarial_attack = pgd
    )
trainer.run_validation_loop_only()

Val Loop:
Batch: 0 of 91. Loss: 1.0866955518722534. Mean so far: 1.0866955518722534. Mean of 100: 1.0866955518722534
Batch: 10 of 91. Loss: 1.1459988355636597. Mean so far: 1.1411862210793928. Mean of 100: 1.1411862210793928
Batch: 20 of 91. Loss: 1.2544273138046265. Mean so far: 1.1389001522745406. Mean of 100: 1.1389001522745406
Batch: 30 of 91. Loss: 1.2419512271881104. Mean so far: 1.1697880202724087. Mean of 100: 1.1697880202724087
Batch: 40 of 91. Loss: 1.398044228553772. Mean so far: 1.193270495752009. Mean of 100: 1.193270495752009
Batch: 50 of 91. Loss: 1.2761154174804688. Mean so far: 1.2179896819825267. Mean of 100: 1.2179896819825267
Batch: 60 of 91. Loss: 1.3517777919769287. Mean so far: 1.2345125939025254. Mean of 100: 1.2345125939025254
Batch: 70 of 91. Loss: 0.9303495287895203. Mean so far: 1.2245079707091964. Mean of 100: 1.2245079707091964
Batch: 80 of 91. Loss: 1.0283088684082031. Mean so far: 1.1987371533005327. Mean of 100: 1.1987371533005327
Batch: 90 of 91. Loss:

In [19]:
pgd = ProjectedGradientDescent(resnet, loss_fn, iterations = 5, alpha = 8/255, epsilon = 4/255, return_logits=False)
trainer = Trainer(
        model = resnet,
        loss_fn = loss_fn,
        classes = classes,
        num_epochs = 10,
        model_name = f'pascal_voc_resnet',
        training_params = training_params,
        multi_label = True,
        print_frequency = 10,
        adversarial_training = True,
        adversarial_attack = pgd
    )
trainer.run_validation_loop_only()

Val Loop:
Batch: 0 of 91. Loss: 0.8559137582778931. Mean so far: 0.8559137582778931. Mean of 100: 0.8559137582778931
Batch: 10 of 91. Loss: 0.9636131525039673. Mean so far: 0.9094706611199812. Mean of 100: 0.9094706611199812
Batch: 20 of 91. Loss: 1.0703169107437134. Mean so far: 0.9257149213836307. Mean of 100: 0.9257149213836307
Batch: 30 of 91. Loss: 0.9562279582023621. Mean so far: 0.9501919323398221. Mean of 100: 0.9501919323398221
Batch: 40 of 91. Loss: 1.2231591939926147. Mean so far: 0.9775824037993827. Mean of 100: 0.9775824037993827
Batch: 50 of 91. Loss: 1.0856913328170776. Mean so far: 1.0039467589527953. Mean of 100: 1.0039467589527953
Batch: 60 of 91. Loss: 1.1683577299118042. Mean so far: 1.0239541696720436. Mean of 100: 1.0239541696720436
Batch: 70 of 91. Loss: 0.7361401915550232. Mean so far: 1.0122528697403383. Mean of 100: 1.0122528697403383
Batch: 80 of 91. Loss: 0.8312792181968689. Mean so far: 0.9902782396033958. Mean of 100: 0.9902782396033958
Batch: 90 of 91. Lo

In [20]:
trainer = Trainer(
        model = resnet,
        loss_fn = loss_fn,
        classes = classes,
        num_epochs = 10,
        model_name = f'pascal_voc_resnet',
        training_params = training_params,
        multi_label = True,
        print_frequency = 10
    )
trainer.run_validation_loop_only()

Val Loop:
Batch: 0 of 91. Loss: 0.07475318014621735. Mean so far: 0.07475318014621735. Mean of 100: 0.07475318014621735
Batch: 10 of 91. Loss: 0.07176012545824051. Mean so far: 0.08431691642511975. Mean of 100: 0.08431691642511975
Batch: 20 of 91. Loss: 0.12300653755664825. Mean so far: 0.08212617376730555. Mean of 100: 0.08212617376730555
Batch: 30 of 91. Loss: 0.10914786905050278. Mean so far: 0.094605348884098. Mean of 100: 0.094605348884098
Batch: 40 of 91. Loss: 0.06766491383314133. Mean so far: 0.08991161543058186. Mean of 100: 0.08991161543058186
Batch: 50 of 91. Loss: 0.09842188656330109. Mean so far: 0.09635884000682364. Mean of 100: 0.09635884000682364
Batch: 60 of 91. Loss: 0.06598218530416489. Mean so far: 0.09403371120818325. Mean of 100: 0.09403371120818325
Batch: 70 of 91. Loss: 0.04896404966711998. Mean so far: 0.09100739883495049. Mean of 100: 0.09100739883495049
Batch: 80 of 91. Loss: 0.06958677619695663. Mean so far: 0.08851808571705112. Mean of 100: 0.08851808571705